In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile

# Đường dẫn tới thư mục chứa các tệp ZIP
zip_folder_path = '/content/drive/MyDrive/AI_Challenge_2023/Data/key_frame_raw'

# Đường dẫn đích cho các tệp giải nén
destination_folder = '/content/drive/MyDrive/AI_Challenge_2023/Data'

# Lặp qua tất cả các tệp ZIP trong thư mục
for file_name in os.listdir(zip_folder_path):
    if file_name.endswith('.zip'):
        zip_file_path = os.path.join(zip_folder_path, file_name)

        # Giải nén tệp ZIP
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(destination_folder)

print('Đã giải nén tất cả các tệp ZIP thành công!')

Đã giải nén tất cả các tệp ZIP thành công!


In [ ]:
import zipfile
import shutil
import os

folder_path = '/content/drive/MyDrive/AI_Challenge_2023/Data/key_frame_raw'  # Đường dẫn đến thư mục chứa các tệp tin zip
extract_path = '/content'  # Đường dẫn đến thư mục để giải nén và lưu các tệp tin

# Tạo thư mục để lưu các tệp tin đã giải nén
os.makedirs(extract_path, exist_ok=True)

# Lặp qua các tệp tin zip trong thư mục
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if filename.endswith('.zip'):
        # Giải nén tệp tin
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

# Di chuyển thư mục đã giải nén vào Google Drive
# Thay đổi đường dẫn đến thư mục trong Google Drive của bạn

print("Giải nén và lưu tệp tin thành công!")

Giải nén và lưu tệp tin thành công!


In [ ]:
import zipfile

with zipfile.ZipFile('file.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zip:
    zip.write('/content/drive/MyDrive/keyframes')

# **CLIP model**

In [ ]:
!pip install clip

In [ ]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import torch
import clip
from PIL import Image

# Load mô hình CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Chuẩn bị ảnh và văn bản
image_path = "/content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/L01_V001/0020.jpg"
texts = ["abc", "def", "ghi"]  # Danh sách các văn bản tham chiếu

# Xử lý ảnh
image = Image.open(image_path)
image_input = preprocess(image).unsqueeze(0).to(device)

# Tạo embedding cho ảnh
with torch.no_grad():
    image_features = model.encode_image(image_input)

# Tính toán độ tương đồng giữa ảnh và từng văn bản
for text in texts:
    text_input = clip.tokenize([text]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_input)
        similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    print(f"Similarity with '{text}': {similarity.item()}")

Similarity with 'abc': 1.0
Similarity with 'def': 1.0
Similarity with 'ghi': 1.0


In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("/content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/L01_V001/0003.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["1 woman",'two people','cat']).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)

Label probs: [[0.246   0.7344  0.01958]]


In [ ]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image
import clip

# Load the pre-trained CLIP model
model, preprocess = clip.load("ViT-B/32", device="cuda")

# Set the path to the folder containing images
image_folder = "/content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/L01_V001"

# Set the text description
text = "a woman"

# Preprocess the text
text_input = clip.tokenize([text]).to("cuda")

# Calculate image features for all images in the folder
image_features = []
image_paths = []

for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    image = Image.open(image_path).convert("RGB")
    image_input = preprocess(image).unsqueeze(0).to("cuda")
    image_features.append(model.encode_image(image_input))
    image_paths.append(image_path)

image_features = torch.cat(image_features, dim=0)

# Calculate similarity scores
with torch.no_grad():
    text_features = model.encode_text(text_input)
    similarity_scores = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# Sort images based on similarity scores
sorted_indices = similarity_scores.argsort(descending=True)

# Print top 5 matching images
top_k = 5
for i in range(top_k):
    image_index = sorted_indices[i]
    image_path = image_paths[image_index]
    similarity_score = similarity_scores[image_index]
    print(f"Image {i+1}: {image_path} (Similarity: {similarity_score.item()})")

Image 1: /content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/L01_V001/0175.jpg (Similarity: 1.0)
Image 2: /content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/L01_V001/0175.jpg (Similarity: 1.0)
Image 3: /content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/L01_V001/0175.jpg (Similarity: 1.0)
Image 4: /content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/L01_V001/0175.jpg (Similarity: 1.0)
Image 5: /content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/L01_V001/0175.jpg (Similarity: 1.0)


In [ ]:
!pip install dalle_pytorch
!pip install --upgrade pytorch-lightning
!pip show pytorch-lightning

In [ ]:
!pip install pycocoevalcap

# **BLIP model**

In [6]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00


In [7]:
#Load BlIP model
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

In [ ]:
# img_url = '/content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/L01_V001/0003.jpg'
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

img_path = '/content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/L01_V001/0001.jpg'
raw_image = Image.open(img_path).convert('RGB')

# conditional image captioning
# text = "a photography of"
# inputs = processor(raw_image, text, return_tensors="pt").to("cuda")

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))

# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a close up of a logo with a city in the background


In [ ]:
import os
list_csv = os.listdir('/content/drive/MyDrive/AI_Challenge_2023/Data/map-keyframes')
list_csv.sort()
for i in list_csv :
  print(i)

In [ ]:
import os
import pandas as pd

list_csv = os.listdir('/content/drive/MyDrive/AI_Challenge_2023/Data/map-keyframes')
list_csv.sort()
for i in list_csv :
    temp = []
    df = pd.read_csv('/content/drive/MyDrive/AI_Challenge_2023/Data/map-keyframes/' + i)
    path_keyframes = '/content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/' +  i.replace(".csv", "") + "/"
    list_keyframes = os.listdir(path_keyframes)
    list_keyframes.sort()
    for image_name in list_keyframes:
        img_path = path_keyframes + image_name
        # print(image_name)
        raw_image = Image.open(img_path).convert('RGB')
        inputs = processor(raw_image, return_tensors="pt").to("cuda")
        out = model.generate(**inputs)
        temp.append(processor.decode(out[0], skip_special_tokens=True))
        # print(processor.decode(out[0], skip_special_tokens=True))
    print(i)
    df['Caption'] = temp
    df.to_csv('/content/drive/MyDrive/AI_Challenge_2023/Data/map-keyframes/' + i, index=False)

In [ ]:
import os
import pandas as pd
from PIL import Image

# Đường dẫn tới thư mục chứa file CSV
csv_folder = "/content/drive/MyDrive/AI_Challenge_2023/Data/map-keyframes/"

# Đường dẫn tới thư mục chứa keyframes
keyframes_folder = "/content/drive/MyDrive/AI_Challenge_2023/Data/keyframes/"

# Lấy danh sách các file CSV
csv_files = os.listdir(csv_folder)

for csv_file in csv_files:
    # Lấy tên file CSV
    csv_filename = os.path.join(csv_folder, csv_file)

    # Lấy tên thư mục keyframes tương ứng
    keyframes_folder_name = os.path.splitext(csv_file)[0]

    # Lấy đường dẫn tới thư mục keyframes
    keyframes_path = os.path.join(keyframes_folder, keyframes_folder_name)

    # Kiểm tra nếu thư mục keyframes tồn tại
    if os.path.isdir(keyframes_path):
        # Đọc file CSV vào DataFrame
        df = pd.read_csv(csv_filename)

        temp = []

        # Lấy danh sách các file keyframe trong thư mục
        keyframe_files = os.listdir(keyframes_path)

        for image_name in keyframe_files:
            img_path = os.path.join(keyframes_path, image_name)
            raw_image = Image.open(img_path).convert('RGB')
            inputs = processor(raw_image, return_tensors="pt").to("cuda")
            out = model.generate(**inputs)
            temp.append(processor.decode(out[0], skip_special_tokens=True))

        # Gán giá trị vào cột 'Caption' của DataFrame
        df['Caption'] = temp

        # Lưu lại DataFrame vào file CSV ban đầu
        df.to_csv(csv_filename, index=False)

In [ ]:
#gg translate
import requests
from urllib.parse import urlencode
def translate(lang_from, lang_to, text):
    base_url = "https://script.google.com/macros/s/AKfycbw6weFFX7Pc1iR0Jh-Z1cH3dZwNKYHFIFploP-kPbS0PvQvh_KofRfQpQtRKOlQUlN3gA/exec"
    params = {
        "q": text,
        "target": lang_to,
        "source": lang_from
    }
    url = base_url + "?" + urlencode(params)
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(url, headers=headers)
    return response.text

text = "Chào thế giới"
print("Translated text: " + translate("vi", "en", text))

Translated text: Hello world


# **Tính toán độ tương đồng giữa 2 đoạn text**

In [ ]:
!pip install pycocoevalcap

In [13]:
import nltk
nltk.download('stopwords')  # Tải xuống tài nguyên stopwords
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
import os
import pandas as pd

# Thời điểm bắt đầu
start_time = time.time()

def preprocess_text(text):
    # Tách từ và loại bỏ stop words
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return tokens

def calculate_cosine_similarity(text1, text2):
    # Tiền xử lý và biểu diễn hai đoạn văn bản thành các vectơ
    tokens1 = preprocess_text(text1)
    tokens2 = preprocess_text(text2)
    all_tokens = list(set(tokens1 + tokens2))
    vector1 = [tokens1.count(token) for token in all_tokens]
    vector2 = [tokens2.count(token) for token in all_tokens]

    # Tính toán cosine similarity
    dot_product = sum(i*j for i, j in zip(vector1, vector2))
    magnitude1 = sum(i**2 for i in vector1) ** 0.5
    magnitude2 = sum(i**2 for i in vector2) ** 0.5
    cosine_similarity = dot_product / (magnitude1 * magnitude2)
    return cosine_similarity

# Đoạn văn bản mẫu
# text1 = "two people standing next to each other"
# text2 = "a men standing next to the a momen"
# Tính toán cosine similarity
# cosine_sim = calculate_cosine_similarity(text1, text2)
cosine_sim = []
text_input = "a person wearing glasses"
i = 1
list_csv = os.listdir('/content/drive/MyDrive/AI_Challenge_2023/Data/map-keyframes')
list_csv.sort()
# Duyệt qua từng tệp tin trong thư mục
for csv_name in list_csv:
    df = pd.read_csv('/content/drive/MyDrive/AI_Challenge_2023/Data/map-keyframes/'+csv_name)
    for cap in df['Caption'] :
      cosine_sim.append(calculate_cosine_similarity(text_input, cap))
    i += 1
    if i == 2 : break


print("Cosine Similarity:", len(cosine_sim))
# Thời điểm kết thúc
end_time = time.time()

# Tính thời gian chạy
execution_time = end_time - start_time
print(cosine_sim[0])
print("Thời gian chạy:", execution_time, "giây")

Cosine Similarity: 236
0.0
Thời gian chạy: 0.11702132225036621 giây


In [18]:
max_value = max(cosine_sim)  # Tìm giá trị lớn nhất trong list
max_index = cosine_sim.index(max_value)  # Lấy số thứ tự của giá trị lớn nhất

print("Giá trị lớn nhất:", max_value)
print("Số thứ tự của giá trị lớn nhất:", max_index)

Giá trị lớn nhất: 0.5773502691896258
Số thứ tự của giá trị lớn nhất: 147


In [17]:
# Sử dụng list comprehension để tạo ra một list các cặp (giá trị, số thứ tự)
value_index_pairs = [(value, index) for index, value in enumerate(cosine_sim)]

# Sắp xếp list các cặp theo giá trị giảm dần
value_index_pairs.sort(reverse=True)
# In ra số thứ tự của 10 giá trị lớn nhất
for value, index in value_index_pairs[:10]:
    print("Giá trị:", value, " - Số thứ tự:", index)

Giá trị: 0.5773502691896258  - Số thứ tự: 152
Giá trị: 0.5773502691896258  - Số thứ tự: 151
Giá trị: 0.5773502691896258  - Số thứ tự: 149
Giá trị: 0.5773502691896258  - Số thứ tự: 147
Giá trị: 0.2581988897471611  - Số thứ tự: 217
Giá trị: 0.2581988897471611  - Số thứ tự: 137
Giá trị: 0.2581988897471611  - Số thứ tự: 38
Giá trị: 0.23570226039551587  - Số thứ tự: 216
Giá trị: 0.23570226039551587  - Số thứ tự: 145
Giá trị: 0.23570226039551587  - Số thứ tự: 18
